In [1]:
import os
import glob
from Bio import Entrez
import pandas as pd
from datetime import date
from datetime import datetime
import re

## Find the HiChIP_Database Google Sheet (Old) and the Newly Queried Excel Sheet (New)

In [19]:
# make an output directory
outdir = 'results/hichip_db/'
os.makedirs(outdir, exist_ok=True)

# get a list of filenames containing 'HiChIP_Databases' and another containing 'Query' 
cwd = os.getcwd()
Query_files = []
os.chdir(cwd)

glob_path = os.path.join(outdir, '*.xlsx')
for file in glob.glob(glob_path):
        
    if 'Query'.casefold() in file.casefold():
        Query_files.append(file)

# sort the two lists in descending order
Query_files.sort(key=str.lower, reverse=True)
old_file = cwd + "/" + Query_files[1]
new_file = cwd + "/" + Query_files[0]

In [8]:
# get the newest and second newest paper tables
old_df = pd.read_excel(old_file)
new_df = pd.read_excel(new_file)

##  Compare Old and New GEO IDs

In [9]:
# make a regex to extract GEO ID
GEO_pattern = re.compile('GSE[0123456789]+')

# make a set of old GEO IDs
old_GEOs = set()
for x in old_df['GEO / Data link'].tolist():
    GEO = GEO_pattern.findall(x)
    old_GEOs.update(GEO)

# make a set of new GEO IDs
new_GEOs = set()
for x in new_df['GEO / Data link'].tolist():
    GEO = GEO_pattern.findall(x)
    new_GEOs.update(GEO)

# compare the sets of GEO IDs
differences_GEOs = new_GEOs.difference(old_GEOs)

In [10]:
# create an indicate column of papers whose GEO ID is not in the old table 
bools = []
for geoid_strings in new_df['GEO / Data link'].tolist():
    
    geo_status = False
    for geoid in geoid_strings.split():
        if geoid in differences_GEOs:
            geo_status = True
            break
    bools.append(geo_status)

differences_df = new_df.loc[bools]

# replace na with empty string
differences_df.fillna("", inplace=True)

/tmp/ipykernel_63603/1355104263.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  differences_df.fillna("", inplace=True)


In [11]:
differences_df

,Paper Title,DOI,Journal,Authors,Year,GEO / Data link,Any other information,Organism,Tissue/Cell Line,Potential HiChIP,Other matched data,Presenter,Date Added
0,A primed immune transcriptional program is act...,10.1016/j.neuron.2021.12.034,Neuron,Meijer et al.,2021,GSE166179,Genome binding/occupancy profiling by high thr...,Mus musculus; Homo sapiens,,Yes,GSM5065201: H3K27ac_HiChIP_IFN_mOPC_rep2\nGSM5...,,<class 'datetime.date'>
1,A small set of accessible enhancers enables br...,10.1093/nar/gkab1125,Nucleic Acids Res,Zaurin et al.,2021,GSE179666,Expression profiling by high throughput sequen...,Homo sapiens,,Yes,GSM5425946: aHiChIP PR T0\nGSM5425945: aHiChIP...,,<class 'datetime.date'>
2,BCL11B enhancer hijacking defines a subtype of...,10.1158/2159-8290.CD-21-0145,Cancer Discov,Montefiori et al.,2021,GSE165209\nGSE165207,Genome binding/occupancy profiling by high thr...,Homo sapiens,,Yes,GSM5028231: Jurkat HiChIP\nGSM5028228: SJMPAL0...,,<class 'datetime.date'>
3,CTCF HiChIP data in A673 Ewing sarcoma cell li...,10.1016/j.ccell.2021.04.001,Cancer Cell,Surdez et al.,2021,GSE156650,Other,Homo sapiens,,Yes,GSM4735773: A673_siCT-Dh1_72h_HiChIP_CTCF_Rep1...,,<class 'datetime.date'>
5,Cohesin protein STAG2 regulates expression of ...,,,,2022,GSE186055\nGSE156773\nGSE156772\nGSE156771\nGS...,Other\nExpression profiling by high throughput...,Homo sapiens,,Maybe,GSM5629821: KD\nGSM5629820: WT\nGSM4743725: M1...,,<class 'datetime.date'>
6,Dynamic chromatin regulatory landscape of huma...,10.1073/pnas.2104758118,Proc Natl Acad Sci U S A,Gennert et al.,2021,GSE168881,Other,Homo sapiens,,Maybe,GSM5171871: Day10_CD19_H3K27ac_rep2\nGSM517186...,,<class 'datetime.date'>
7,EBF1 nuclear repositioning instructs chromatin...,10.1016/j.molcel.2022.01.015,Mol Cell,Zhou et al.,2022,GSE173872\nGSE173871\nGSE173843,Expression profiling by high throughput sequen...,Homo sapiens,,Yes,GSM5281678: MB157pInd20EBF1_10dox_SMC1_HiChIP\...,,<class 'datetime.date'>
8,EWSR1-ATF1 dependent 3D connectivity regulates...,,,,2022,GSE180198\nGSE180194,Genome binding/occupancy profiling by high thr...,Homo sapiens,,Yes,GSM5455281: SUCCS1_siEA_96h_HiChIP\nGSM5455278...,,<class 'datetime.date'>
9,Epigenomics landscape of colorectal cancer,10.1136/gutjnl-2020-322835,Gut,Orouji et al.,2021,GSE136889,Genome binding/occupancy profiling by high thr...,Homo sapiens,,Yes,GSM4053398: LS513 H3K27ac HiChIP\nGSM4053399: ...,,<class 'datetime.date'>
10,Epstein-Barr virus nuclear antigen 2 (EBNA2) e...,10.1101/gr.264705.120,Genome Res,Hong et al.,2021,GSE179755\nGSE178131\nGSE177046\nGSE176421\nGS...,Other\nGenome variation profiling by SNP array...,Homo sapiens,,Yes,GSM5431873: HiChIP: EBNA2- (P3HR1) EBV infecte...,,<class 'datetime.date'>


## Save the Output File

In [3]:
# determining the current year, month and day
today = date.today()
date_str = today.strftime("%Y_%m_%d")

# determining current hour and minute
now = datetime.now()
time_str = now.strftime("%H_%M")

# setting the output filename
output = os.path.join(outdir, "GEO_Compare.{}_{}".format(date_str, time_str))

In [12]:
differences_df.to_excel(output+".xlsx", index=False)